<a href="https://colab.research.google.com/github/ialimustufa/Gradient-Centralization-TensorFlow/blob/main/examples/gctf_horses_v_humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCTF Horses vs Humans

This notebook shows the the process of using the [`gradient-centralization-tf`](https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow) Python package to train on the [Horses vs Humans](http://www.laurencemoroney.com/horses-or-humans-dataset/) dataset by [Laurence Moroney](https://twitter.com/lmoroney). Gradient Centralization is a simple and effective optimization technique for Deep Neural Networks as suggested by Yong et al. in the paper 
[Gradient Centralization: A New Optimization Technique for Deep Neural Networks](https://arxiv.org/abs/2004.01461). It can both speedup training 
 process and improve the final generalization performance of DNNs.

If you find this useful please consider giving a ⭐ to [the repo](https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow/).

## A bit about GC

Gradient Centralization operates directly on gradients by centralizing the gradient vectors to have zero mean. It can both speedup training process and improve the final generalization performance of DNNs. Here is an Illustration of the GC operation on gradient matrix/tensor of weights in the fully-connected layer (left) and convolutional layer (right). GC computes the column/slice mean of gradient matrix/tensor and centralizes each column/slice to have zero mean.

![](https://i.imgur.com/KitoO8J.png)

GC can be viewed as a projected gradient descent method with a constrained loss function. The geometrical interpretation of GC. The gradient is projected on a hyperplane $e^T(w-w^t)=0$, where the projected gradient is used to update the weight.

![](https://i.imgur.com/ekHhQv0.png)

## Setup

In [1]:
import tensorflow as tf
from time import time

### Install the package


In [2]:
!pip install gradient-centralization-tf

## Get the data

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip
  
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

--2021-02-21 12:08:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   262MB/s    in 0.5s    

2021-02-21 12:08:31 (262 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2021-02-21 12:08:32--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.144, 142.250.73.208, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

## Image Augmentation

We will perform a couple of augmentations on the image

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Training the model

Here we have built a very simple model with 5 Convolutional for this example. 

In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

On the same note since we are interested in comparing results we will create a callback which allows us to compute the training time.

In [6]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)

### Train a model without [`gctf`](https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow/)

In [7]:
from tensorflow.keras.optimizers import RMSprop

time_callback_no_gctf = TimeHistory()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

history_no_gctf = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,
      callbacks = [time_callback_no_gctf])

Epoch 1/10
8/8 [==============================] - 27s 2s/step - loss: 0.7229 - accuracy: 0.4852 - val_loss: 0.6898 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 22s 3s/step - loss: 0.6871 - accuracy: 0.5560 - val_loss: 0.6858 - val_accuracy: 0.5234
Epoch 3/10
8/8 [==============================] - 22s 3s/step - loss: 0.6732 - accuracy: 0.6040 - val_loss: 0.6801 - val_accuracy: 0.5508
Epoch 4/10
8/8 [==============================] - 22s 3s/step - loss: 0.6343 - accuracy: 0.6694 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 5/10
8/8 [==============================] - 21s 3s/step - loss: 0.6548 - accuracy: 0.6131 - val_loss: 0.6718 - val_accuracy: 0.8281
Epoch 6/10
8/8 [==============================] - 21s 3s/step - loss: 0.5896 - accuracy: 0.6966 - val_loss: 0.6733 - val_accuracy: 0.5000
Epoch 7/10
8/8 [==============================] - 21s 3s/step - loss: 0.5870 - accuracy: 0.7058 - val_loss: 0.6604 - val_accuracy: 0.6094
Epoch 8/10
8/8 [==================

### Train a model with [`gctf`](https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow/)

In [8]:
import gctf #import gctf

time_callback_gctf = TimeHistory()
model.compile(loss='binary_crossentropy',
              optimizer=gctf.optimizers.rmsprop(learning_rate = 1e-4),
              metrics=['accuracy'])

history_gctf = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,
      callbacks = [time_callback_gctf])

Epoch 1/10
8/8 [==============================] - 24s 3s/step - loss: 0.6394 - accuracy: 0.6779 - val_loss: 0.6885 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 21s 3s/step - loss: 0.5504 - accuracy: 0.7124 - val_loss: 0.6450 - val_accuracy: 0.5625
Epoch 3/10
8/8 [==============================] - 22s 3s/step - loss: 0.5050 - accuracy: 0.7673 - val_loss: 0.6163 - val_accuracy: 0.6094
Epoch 4/10
8/8 [==============================] - 21s 3s/step - loss: 0.5206 - accuracy: 0.7589 - val_loss: 0.5969 - val_accuracy: 0.6797
Epoch 5/10
8/8 [==============================] - 21s 3s/step - loss: 0.5175 - accuracy: 0.7506 - val_loss: 0.7745 - val_accuracy: 0.5000
Epoch 6/10
8/8 [==============================] - 23s 3s/step - loss: 0.6449 - accuracy: 0.6996 - val_loss: 0.6114 - val_accuracy: 0.5820
Epoch 7/10
8/8 [==============================] - 21s 3s/step - loss: 0.5059 - accuracy: 0.7551 - val_loss: 0.5494 - val_accuracy: 0.7461
Epoch 8/10
8/8 [==================

## Compare results

In this example we are further interested in also comparing the results

In [9]:
from tabulate import tabulate

data = [["Model without gctf:",sum(time_callback_no_gctf.times),history_no_gctf.history['accuracy'][-1],history_no_gctf.history['loss'][-1]],
        ["Model with gctf",sum(time_callback_gctf.times),history_gctf.history['accuracy'][-1],history_gctf.history['loss'][-1]]] 

print(tabulate(data, headers=["Type","Execution time", "Accuracy", "Loss"]))

Type                   Execution time    Accuracy      Loss
-------------------  ----------------  ----------  --------
Model without gctf:           221.626    0.690768  0.625912
Model with gctf               216.744    0.805339  0.426568
